## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [3]:
soup = BeautifulSoup(res.content,'lxml')

In [4]:
table = soup.find_all('table')[0]

In [5]:
df = pd.read_html(str(table))[0]

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
df = df[df['Borough'] != 'Not assigned']

In [8]:
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
for i, row in df.iterrows():
    if (row['Borough'] != 'Not assigned') & (row['Neighbourhood'] == 'Not assigned'):
        df.loc[i,'Neighbourhood'] = df.loc[i,'Borough']

#### More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [10]:
df = df.groupby(['Postcode', 'Borough']).agg(lambda x: tuple(x)).applymap(list)
df = df.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [11]:
for index, row in df.iterrows():
    elements = len(row['Neighbourhood'])
    df.at[index,'Neighbourhood2'] = ''
    for i in range(elements):
        df.at[index,'Neighbourhood2'] = df.at[index,'Neighbourhood2'] + row['Neighbourhood'][i] + ', '
    df.loc[index,'Neighbourhood2'] = df.loc[index,'Neighbourhood2'][:-2]

In [12]:
df = df.drop('Neighbourhood',axis=1)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df.shape

(103, 3)

### Use Geocoder to get the latitude and longitude of all the postal codes

In [14]:
!pip install geocoder

     |████████████████████████████████| 102kB 15.2MB/s ta 0:00:01


In [15]:
import geocoder
for index, row in df.iterrows():
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(row.Postcode))
        lat_lng_coords = g.latlng  
    df.at[index,'Latitude'] = lat_lng_coords[0]
    df.at[index,'Longitude'] = lat_lng_coords[1]

In [225]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
